## 110-2 Operations Research Case 2

Algorithm 4: based on Proc time/LST Ratio

Add find-hole feature (consider scheduling to the 'idleness', that is, the holes first)



## Loading data

In [35]:
import pandas as pd 
import numpy as np

In [38]:
datadir = './data'
instances = []
for i in range(5):
    name = f'instance_{i+1}.csv'
    fullpath = datadir+'/'+name
    instances.append(pd.read_csv(fullpath))

In [39]:
GAP = 1e-7

## Data Structures

In [40]:
# job structure 
class Job:
    '''structure for 1 job '''
    def __init__(self, row):
        '''input := df.iloc[idx, :]'''
        self.id = row['Job ID']
        self.due = row['Due Time']
        self.next_op = 0 # True as complete, False as not yet processed
        self.stage_pt = [row['Stage-1 Processing Time'], row['Stage-2 Processing Time']]
        mfor1 = list(map(int, row['Stage-1 Machines'].split(',')))
        if row['Stage-2 Machines'] is not np.nan:
            mfor2 = list(map(int, row['Stage-2 Machines'].split(',')))
        else: mfor2 = [] 
        self.stage_mach = [mfor1, mfor2]
        self.assign_mach = [None for _ in range(2)]
        self.start_time = [-1 for _ in range(2)]
        self.end_time = [-1 for _ in range(2)]
    
    def __repr__(self):
        return f'\
          * Job id: {self.id}\n\
          * Due time:{self.due}\n\
          stage 1: {self.assign_mach[0]}\n\
                   {self.stage_pt[0]}, {self.stage_mach[0]}\n\
          stage 2: {self.assign_mach[1]}\n\
                   {self.stage_pt[1]}, {self.stage_mach[1]}'
    __str__ = __repr__

In [146]:
class Jobs:
    '''structure for multiple jobs' management'''
    def __init__(self, n):
        self.completion_times = np.zeros(n)
        self.tardiness = np.zeros(n)
        self.is_completed = np.full(n, False)
        self.residual_times = np.zeros(n)
        self.jobs = []
        
    def get_RRDD(self):
        if getattr(self, 'RRDD', None) is None:
            self.RRDD = self.due_dates - np.min(self.due_dates)
        return self.RRDD # static

    def get_LST(self):
        '''latest start time'''
        self.LST = self.due_dates - self.residual_times
        return self.LST
    
    def add_jobs(self, data):
        self.due_dates = data['Due Time'].to_numpy()
        for i in range(len(data)):
            row = data.iloc[i, :]
            jobi = Job(row)
            self.residual_times[i] = sum(jobi.stage_pt)
            self.jobs.append(jobi)
            
    
    def assign(self, job_name, mach, st):
        '''job_name = (2, 0) means job 3 and op 1
        note that job and op is 0-indexed as well as machines
        op
        '''
        
        i = 0 
        jobidx, op = job_name 
        job = self.jobs[jobidx]
        J.completion_times[jobidx] = st + job.stage_pt[op]
        J.residual_times[jobidx] -= job.stage_pt[op]
        job.assign_mach[op] = mach
        job.start_time[op] = st
        job.end_time[op] = J.completion_times[jobidx]
        job.next_op = op+1
        if op == 1:
            self.is_completed[jobidx] = True

In [322]:
class Machines:
    def __init__(self, df):
        '''pass the stage1, stage2 machine lists'''
        mfor1 = df['Stage-1 Machines'].values.tolist()
        mfor2 = df['Stage-2 Machines'].values.tolist()
        mfor1 = [list(map(int, x.split(','))) for x in mfor1]
        mfor2 = [list(map(int, x.split(','))) for x in mfor2 if x is not np.nan]
        mfor1 = [item for sublist in mfor1 for item in sublist]
        mfor2 = [item for sublist in mfor2 for item in sublist]
    
        self.number = max(max(mfor1), max(mfor2))
        self.versatile = [mfor1.count(i+1) + mfor2.count(i+1) for i in range(self.number)]
        self.holes = [[] for _ in range(self.number)]
        
        self.fintime = [0 for _ in range(self.number)]
        self.HL = [0 for _ in range(self.number)]
        
        
    def _schedule(self, mach, job_name, st, proc_time):
        '''mach is 0-indexed'''
        display_name = tuple([x+1 for x in job_name])
        end_time = st + proc_time
        # self.schedule[mach].append((f'{display_name}', round(end_time, 3))) 
        self.fintime[mach] = st + proc_time
    
    def add_idle(self, mach, idle_time,
                hole_start, hole_end):
        # self.schedule[mach].append((f'idle', round(hole_end)))
        self.fintime[mach] += idle_time
        
        
        # add hole 
        # len(self.schedule)-1 is the idx of this hole in schedule
        hole = tuple([hole_start, hole_end])
        self.holes[mach].append(hole)
        self.HL[mach] += hole_end - hole_start
        
    def schedule_hole(self, job_name, mach, hole_id, fill_end):
        '''update and return updated average hole length'''
        hole = self.holes[mach][hole_id]
        hole_start, hole_end = hole
        hole_length = hole_end - hole_start
        if abs(hole_end - fill_end) < GAP:
            # pop the hole by hole_id 
            self.holes[mach].pop(hole_id)
        else:
            self.holes[mach][hole_id] = tuple([fill_end, hole_end])
        self.HL -= (fill_end - hole_start) # processing time 
        
    
        display_name = tuple([x+1 for x in job_name])
        
        
        return self.HL/len(self.holes)
            
        
        
        

## Algorithm

Updated: 2022.5.5 Thursday 

#### Warnings:

1. Re-run the code from **preprcoessing section** otherwise the data stuctures will keep accumulating repetitive datas. 

2. All the indexing is 0-indexed for coding convenience, but when storing back to `M.schedule` for displaying purpose, it is changed into 1-indexed. 





#### Steps:

- The below steps may be explained (made more clear) by using some notations. 
- For operation, it is the same meaning as stage. 
- `GAP = 1e-7`. 
- Using `find_hole()` method helps reducing makespan in total, and may indrectly contribute to number of tardy job minimization. 


1. ```LST = self.due_dates - self.total_processing_times ```

    `self.due_dates[j]` is the deadline for job ${j}$.
    `total_processing_times[j]` is the total processing times for job ${j}$.
    
    For each stage, we calculate the ${stage\_processing\_time / LST}$ as ${LST\_ratio}$; each operation is associated with a ${LST\_ratio}$. The less LST, the more urgent this operation is; the longer stage_processing_time, the earlier this operation should be started. Therefore, those with **high** ${LST\_ratio}$ should be done first. 
    
2. Jobs are put into a priority queue ${Q}$ ordered by ${LST\_ratio}$. 
   Machines are sorted into ${Mach\_Q}$ ordered by ${versatility}$. 
   ${versatility}$ specifies the number of operations that a machine is capable of executing. For example, 
   in instance_1.csv, the 5 machines' versatility is 9, 22, 22, 22, 22 respectively. 
   The core idea of our heuristic is to schedule job with **high** ${LST\_ratio}$ on low machine with **low** versatility.

3. ```Extract_min()``` from ${Q}$ as `curr_op`. 
    In implementation, we use a min queue, so the ratio are multiplied by ${-1}$ to ensure the ordering. 
    
    3.1. Check if `curr_op` is 2nd stage and processing_time equals to 0; if yes, scheduling it on machine ${None}$ and continue with the next operation extracted from ${Q}$. 
    
    3.2. For every machine, we store a list called `holes`. 
      That is, in order to preserve job operation precedence 
      (op1 must be completed before op2), sometimes op2 must idle until op1 completes, which creates `holes` in the machine's working schedule with various time lengths. 
      To make use of these holes, a `find_hole()` function is called to scan through the machines for an available hole. 
    
    3.3. `find_hole`'s pseudo code:
        Suppose (i, j) (job i, operation j), with processing time $p_{ij}$ is to be scheduled
        for m in Mach_Q:
            // check m's availbility for executing i, j
            if not available((i,j), m):
                continue 
            // check if m has holes, i.e. idle periods on its current schedule
            if not m.holes:
                continue 
            for id, hole in enumerate(m.holes):
                hole_start, hole_end = hole 
                // if j is first op, legal_length = hole_length; 
                // otherwise legal_length is op1's completion time to hole_end 
                if legal_length < p_ij:
                    continue
                else: 
                    // this hole is the hole to be scheduled. 
                    return id, hole
     
     3.4. Schedule `curr_op` into the chosen hole and update associated values. Note that we have to update the `holes` list `m.holes`. If the hole is just the size of `curr_op`'s processing time (with floating precision error with `GAP`), then the hole is popped from the list, if not, the hole is updated to a smaller size. 
     
     3.5. If `find_hole()` succeeds, continue with the next operation, otherwise go to step 4. 
     When it fails, it either means that there are no holes on available machines, or that no holes are with size enough to fit `curr_op` in. 

4. Note that we go to step 4 only when `find_hole()` fails. 
    Here we Calculate the best machine to schedule `curr_op` by getting its associated subset of machines, 
    and then ordering them by (1) their current finished times (for the sake of makespan) and
    (2) their `versatility`. 
    To sum up, the best machine `curr_machine` is computed by: 
    ```curr_machine = min(avail_machines_idx, key = lambda m: (M.fintime[m], M.versatile[m], m))```
   Then put ```curr_op``` onto this machine.

5. We decide if it is a good time to schedule `curr_op`:
    
    Decide if this job is `PERMIT`: 
    
    5.1. if its current completion time plus `curr_op`'s processing time is already tardy, associate it with a very big ${LST\_ratio}$ (eg. `float('inf')`), set `PERMIT = False`. 
    
    5.2. if its scheduling requires an idle time that is too large, postpone it temporarily, associate it with ${LST\_ratio}$ + $k$ where $k >= 0$, set `PERMIT = False`. 
    
    5.3. In both non-permitable cases, increment `failures` to record how many times the job is popped from ${Q}$ but fails to be scheduled. If failures for `curr_op` > `failure_tolerance`, we schedule it no matter waht. `failure_tolerance` is yet another parameter. 
   
    5.4. If `PERMIT`, update values, especially update the completion_times, residual_times and **LST**,
    else, push the operation back to ${Q}$ with new ${LST\_ratio}$.
   
6. Check if all job operations are scheduled, if yes, stop the algorithm, if no, continue the iteration (go back to step 3).

#### Result:

1. `M.fintime` is the finishing time of machines, you can get the makespan by `max(M.fintime)`.
2. `M.schedule` IS REMOVED, use checker to generate it. 
3. `J.completion_times` is the completion times for all jobs. Comparing it with `J.due_dates` using
   ```tardies = list(np.where(J.completion_times > J.due_dates)[0])```
   gives you the tardy jobs (it's 0-indexed!!!). Turn it to 1-indexed by 
   ```[x+1 for x in tardies]```. 
   

## Preprocessing 
Usage:
1. Read the dataframe into `Machines()` as `M`.
2. Init by giving the length of jobs to `Jobs()` as `J`.
    Initialize it by calling `add_jobs()`.
3. Call our heuristic algorithm. 
4. Get the result from `M, J`. No need to return them. 

In [323]:
INSTANCEIDX = 4
df = instances[INSTANCEIDX]
M = Machines(df)
J = Jobs(len(df))
J.add_jobs(df)
# print(J.residual_times)
# print(J.get_LST())

In [340]:
# now put every operation into Q ( Q by operation )
from heapq import heappush, heappop, heapify
def make_Q(J):
    Job_keys = []
    for job_index in range(len(J.jobs)):
        op1_pt, op2_pt = J.jobs[job_index].stage_pt
        # 我覺得 due_dates 幾乎等於 total proc time的反而可以放後面(不太可能來得及，乾脆果斷放棄)
        LST = J.get_LST()
        if LST[job_index] <= GAP: # avoid division by zero error
            lst_ratios = [0, 0]
        else:
            lst_ratios = [op1_pt / J.LST[job_index], op2_pt / J.LST[job_index]]
        # preserve precedence:
        lst_ratios[0] = max(lst_ratios)
        # NOTE: min queue extracts by minimum value, so add a negative sign here
        # (lst_ratio, job_index, job_op)
        Job_keys.append((-lst_ratios[0], job_index, 0))
        Job_keys.append((-lst_ratios[1], job_index, 1))
    Q = Job_keys[:]
    heapify(Q) 
    return Q
Q = make_Q(J)

In [326]:
def getAvailMachs(J, M):
    bigM = M.number 
    bigN = len(J.jobs)
    AvailMachs = np.full((bigN, 2, bigM), 
                         fill_value = False)
    for i in range(bigN):
        mfor1, mfor2 = J.jobs[i].stage_mach[0], J.jobs[i].stage_mach[1]
        for m in mfor1:
            AvailMachs[i][0][m-1] = True
        for m in mfor2:
            AvailMachs[i][1][m-1] = True
    # indexing AvailMachs[i][j][m] to check if job (i,j) can be put on machine m (all ZERO-indexed)
    return AvailMachs

In [327]:
def make_mQ(M):
    Q = []
    for m in range(M.number):
        versatility = M.versatile[m]
        Q.append([versatility, m])
    return sorted(Q, key = lambda x:(x[0], x[1]))

### find_hole

In [328]:
def find_hole(job_name, 
              currjob, 
              J,
              M, 
              Mach_Q, 
              AvailMachTable):
    '''
    https://stackoverflow.com/questions/59903948/how-to-iterate-heapq-without-losing-data
    '''
    curr_jindex, curr_op = job_name 
    proc_time = currjob.stage_pt[curr_op]
    def find_hole_helper():
        for idx, curritem in enumerate(Mach_Q):
            _, m_id = curritem
            if not AvailMachTable[curr_jindex][curr_op][m_id]:
                continue
            # find legal holes
            if not M.holes[m_id]:
                continue
            for hole_id, hole in enumerate(M.holes[m_id]):
                hole_start, hole_end = hole
                hole_length = hole_end - hole_start 
                # enough length 
                if hole_length - proc_time >= -GAP:
                    # check legal precedence 
                    curr_hole_id = hole_id
                    if curr_op == 1 and hole_start >= currjob.end_time[0]:
                        return idx, m_id, curr_hole_id, hole_start + proc_time
                    elif curr_op == 0: 
                        return idx, m_id, curr_hole_id, hole_start + proc_time
                # 一律從hole_start開始schedule，沒辦法的話就跳過（不然更新holes那邊變超麻煩）
    res = find_hole_helper()
    if not res:
        print(f'No result in finding a hole for {[x+1 for x in job_name]}')
        return False
    if res:
        idx, m_id, hole_id, fill_end = res
        print(f'Schduling {m_id+1}, {M.holes[m_id][hole_id]} for {[x+1 for x in job_name]}')
        print(f'Original: {M.holes[m_id]}')
        # idx是Queue中machine的位置
        hole = M.holes[m_id][hole_id]
        print(hole)
        hole_start, hole_end = hole
        J.assign(job_name = job_name, 
                mach = m_id, 
                st = hole_start) 
        # update hole length and replace avg_hole_length 
        new_avg_hl = M.schedule_hole(
                job_name = job_name, 
                mach = m_id, 
                hole_id = hole_id, 
                fill_end = fill_end)  
        print(f'Updated: {M.holes[m_id]}')
        return True 

### heuristic

In [329]:
# while not all operations in all jobs are scheduled

def heuristic(J, M): 
    # best_makepsan = sum(job_processing_time) for all jobs / |M|
    # heperparameters 
    TOLRATIO = 0.3
    Fail_Tolerance = 2
    best_makespan = sum(job.stage_pt[0]+job.stage_pt[1] for job in J.jobs)/M.number
    tolerance = best_makespan * TOLRATIO  # tolerance for idle time, if idle > tolerance, do not schedule the curr op in the current epoch. 
    print(f'[INFO] {len(J.jobs)} jobs, {M.number} machines')
    print(f'[INFO] Tolerance: {tolerance:.2f}')

    # Job_Q (lst_ratio, job_index, job_op) 
    Job_Q = make_Q(J)
    # Mach_Q (versatility, avg_hole_length, m)
    Mach_Q = make_mQ(M)
    AvailMachs = getAvailMachs(J = J, M = M)
    
    
    fails = [0 for _ in range(len(J.jobs))]
    epoch = 0
    while not np.all(J.is_completed):
        epoch += 1
        PERMIT = True
        # step 3. extract_min() to get the job with minimal LST and its other attributes
        _, curr_job_index, curr_op = heappop(Job_Q)
        curr_job = J.jobs[curr_job_index]
        op_proc_time = curr_job.stage_pt[curr_op]
        job_name = (curr_job_index, curr_op)
         
        
        # if curr_job has no second operation 
        if op_proc_time <= 0: 
            J.assign(job_name = job_name, 
                    mach = None,
                    st = curr_job.end_time[curr_op-1]) 
            # note that it's only possible for second operation to have proc time = 0
            # so this doesn't trigger index error
            continue 
        # step 4-1. calculate the best machine: find-hole
        # 'job_name', 'currjob', and 'Mach_Q'
        if find_hole(J = J, M = M,
                     job_name = job_name, currjob = curr_job, Mach_Q = Mach_Q,
                  AvailMachTable = AvailMachs):
            continue
            
    
        # step 4-2. if find-hole fails, calculate the best machine and schedule at the end 
        avail_machines_idx = [x-1 for x in curr_job.stage_mach[curr_op]]
        curr_machine = min(avail_machines_idx, key = lambda x: (M.fintime[x], M.versatile[x], x))
        # ARE THERE REASONS TO POSTPONE THE CURR OP?
        if J.completion_times[curr_job_index] + op_proc_time > J.due_dates[curr_job_index] and fails[curr_job_index] < Fail_Tolerance:
            print(f'[INFO] Job {curr_job_index+1} will be tardy even if scheduled, queue last.')
            curr_new_value = float('inf')
            PERMIT = False
        
        # ARE THERE REASONS TO POSTPONE THE CURR OP?
        elif M.fintime[curr_machine] < J.completion_times[curr_job_index]:
            
            idle = J.completion_times[curr_job_index] - M.fintime[curr_machine]
            if idle > tolerance and curr_op == 1 and fails[curr_job_index] < Fail_Tolerance:
                print(f'[INFO] Job {curr_job_index+1} op {curr_op+1} has idle {idle:.2f}, postpone it.')
                PERMIT = False
                if Q:
                    curr_new_value = Q[0][0] + 3
                else:
                    curr_new_value = 0 # the last one 
            else:
                M.add_idle( 
                hole_start = M.fintime[curr_machine],
                hole_end =  J.completion_times[curr_job_index], 
                mach = curr_machine, 
                idle_time = idle)
         
        if PERMIT:
            print(f'Schduling on machine {curr_machine+1}\'s end {[x+1 for x in job_name]}')
            J.assign(job_name = job_name, 
                mach = curr_machine, 
                 st = M.fintime[curr_machine]
                ) 
            M._schedule(job_name = job_name, 
               mach = curr_machine, 
                proc_time = op_proc_time,
               st = M.fintime[curr_machine])
            curr_new_value = J.get_LST()[curr_job_index]
        else: 
            fails[curr_job_index] += 1
        # print(f'{epoch} Fails Count:', fails)
        # update the LST value and push it back to Q if the job has its second operation that hasn't been done
        if not PERMIT:
            heappush(Job_Q, (curr_new_value, curr_job_index, curr_op))
            # it maintains the heap invariant, no need to heapify

In [330]:
heuristic(J, M)
J.is_completed

[INFO] 20 jobs, 9 machines
[INFO] Tolerance: 5.55
No result in finding a hole for [13, 1]
Schduling on machine 6's end [13, 1]
No result in finding a hole for [13, 2]
[INFO] Job 13 op 2 has idle 7.60, postpone it.
No result in finding a hole for [13, 2]
[INFO] Job 13 op 2 has idle 7.60, postpone it.
No result in finding a hole for [13, 2]
Schduling on machine 2's end [13, 2]
No result in finding a hole for [2, 1]
Schduling on machine 3's end [2, 1]
Schduling 2, (0, 7.6) for [18, 1]
Original: [(0, 7.6)]
(0, 7.6)
Updated: [(5.3, 7.6)]
No result in finding a hole for [4, 1]
Schduling on machine 8's end [4, 1]
No result in finding a hole for [5, 1]
Schduling on machine 1's end [5, 1]
No result in finding a hole for [17, 1]
Schduling on machine 7's end [17, 1]
No result in finding a hole for [17, 2]
[INFO] Job 17 op 2 has idle 9.00, postpone it.
No result in finding a hole for [17, 2]
[INFO] Job 17 op 2 has idle 9.00, postpone it.
No result in finding a hole for [17, 2]
Schduling on machine

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [331]:
A = make_result(J)

In [332]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [333]:
from or_checker import checker, get_machine_number
tardy, mk, schedule = checker(A, instances[INSTANCEIDX])
print('tardy:', tardy, ', makespan', mk)
print(*schedule, sep = '\n')

tardy: [2, 4, 6, 7, 10, 14, 18, 19] , makespan 29.2
[((5, 1), 6.4), ((12, 2), 12.2), ((3, 2), 13.4), ((19, 2), 13.9), ((1, 2), 21.8)]
[((18, 1), 5.3), ((13, 2), 11.9), ((1, 1), 16.8)]
[((2, 1), 9.6), ((16, 1), 15.8), ((20, 1), 22.6)]
[((6, 1), 3.5), ((18, 2), 7.0), ((14, 1), 15.7), ((9, 2), 16.1), ((8, 2), 17.0)]
[((7, 1), 9.6), ((10, 1), 15.0), ((15, 1), 19.8)]
[((13, 1), 7.6), ((6, 2), 12.6), ((11, 1), 16.1), ((20, 2), 29.2)]
[((17, 1), 9.0), ((5, 2), 10.0), ((8, 1), 15.5), ((7, 2), 15.7), ((16, 2), 16.4)]
[((4, 1), 8.9), ((9, 1), 15.5), ((10, 2), 18.1), ((15, 2), 20.4)]
[((3, 1), 4.8), ((12, 1), 5.3), ((19, 1), 8.4), ((17, 2), 15.9), ((11, 2), 18.7)]


In [336]:
Tardy_jobs = list(np.where(J.completion_times > J.due_dates)[0])
Makespan = max(M.fintime)
print(f'Instance {i+1}:')
print('First objective (# tardy):', len(Tardy_jobs), Tardy_jobs)
print('Second objective (makespan):', Makespan)

Instance 5:
First objective (# tardy): 8 [2, 4, 6, 7, 10, 14, 18, 19]
Second objective (makespan): 29.200000000000003


## Testing all instances

In [339]:
Results = []
for inst in range(5):
    data = instances[inst]
    M = Machines(data)
    J = Jobs(len(data))
    J.add_jobs(data)
    heuristic(J = J, M = M)
    print(f'** Summary \ninstance {inst+1}:')
    Tardy_jobs = list(np.where(J.completion_times > J.due_dates)[0])
    Tardy_jobs = [x+1 for x in Tardy_jobs]
    Makespan = max(M.fintime)
    print('First objective (# tardy):', len(Tardy_jobs), Tardy_jobs)
    print('Second objective (makespan):', Makespan)
    print('==================================')
    Results.append({'J':J, 'M':M})

[INFO] 12 jobs, 5 machines
[INFO] Tolerance: 1.81
No result in finding a hole for [1, 1]
Schduling on machine 1's end [1, 1]
No result in finding a hole for [1, 2]
[INFO] Job 1 op 2 has idle 2.70, postpone it.
No result in finding a hole for [1, 2]
[INFO] Job 1 op 2 has idle 2.70, postpone it.
No result in finding a hole for [1, 2]
Schduling on machine 2's end [1, 2]
Schduling 2, (0, 2.7) for [7, 1]
Original: [(0, 2.7)]
(0, 2.7)
Updated: [(1.4, 2.7)]
No result in finding a hole for [7, 2]
Schduling on machine 3's end [7, 2]
No result in finding a hole for [6, 1]
Schduling on machine 4's end [6, 1]
No result in finding a hole for [2, 1]
Schduling on machine 5's end [2, 1]
Schduling 2, (1.4, 2.7) for [3, 1]
Original: [(1.4, 2.7)]
(1.4, 2.7)
Updated: [(2.0999999999999996, 2.7)]
No result in finding a hole for [3, 2]
Schduling on machine 5's end [3, 2]
No result in finding a hole for [2, 2]
Schduling on machine 4's end [2, 2]
No result in finding a hole for [11, 1]
Schduling on machine 1's

In [308]:
see = 4

In [309]:
print(*Results[see]['J'].due_dates, sep = ', ')
print('====')
print(*Results[see]['J'].completion_times, sep = ', ')

21.9, 10.1, 8.9, 10.9, 9.0, 14.4, 14.5, 16.1, 20.4, 19.2, 14.9, 15.2, 12.0, 17.7, 19.0, 18.1, 18.4, 7.7, 10.0, 13.4
====
21.799999999999997, 9.6, 13.399999999999999, 8.9, 10.0, 12.6, 15.7, 16.999999999999996, 16.099999999999998, 18.1, 18.700000000000003, 12.2, 11.899999999999999, 15.7, 20.400000000000002, 16.400000000000002, 15.9, 7.0, 13.899999999999999, 29.200000000000003


In [67]:
instances[1]

,Job ID,Stage-1 Processing Time,Stage-2 Processing Time,Stage-1 Machines,Stage-2 Machines,Due Time
0,1,2.7,1.5,"1,2,3,4,5","2,3,4,5",10
1,2,1.6,2.3,"1,2,3,4,5","2,3,4,5",5
2,3,1.0,2.7,"1,2,3,4,5","2,3,4,5",5
3,4,2.8,0.8,"2,3,4,5","2,3,4,5",5
4,5,0.8,1.9,"1,2,3,4,5","2,3,4,5",5
5,6,2.7,0.0,"1,2,3,4,5",NaN,5
6,7,1.4,1.5,"2,3,4,5","2,3,4,5",5
7,8,2.2,0.0,"2,3,4,5",NaN,10
8,9,0.8,1.8,"1,2,3,4,5","2,3,4,5",10
9,10,2.2,2.2,"2,3,4,5","2,3,4,5",10


## Output

1. Write your heuristic algorithm here.
2. We would call this function in CA2_grading_program.py to evaluate your algorithm.
3. Please do not change the function name and the file name.
4. The parameter is the file path of a data file, whose format is specified in the document. 
5. You need to return your schedule in two lists "machine" and "completion_time".
    (a) machine[j][0] is the machine ID of the machine to process the first stage of job j + 1, and 
machine[j][1] is the machine to process the second stage of job j + 1.
    (b) completion_time[j][0] is the completion time of the first stage of job j + 1, and 
completion_time[j][1] is the completion time of the second stage of job j + 1. 
    
    
    Note 1. If you have n jobs, both the two lists are n by 2 (n rows, 2 columns). 
    Note 2. In the list "machine", you should record the IDs of machines 
            (i.e., to let machine 1 process the first stage of job 1, 
            you should have machine[0][0] == 1 rather than machine[0][0] == 0).
6. You only need to submit this algorithm_module.py.


In [311]:
def make_result(J):
    '''pass in the resulted J'''
    n = len(J.jobs)
    machine, completion_time = [], []
    for i in range(n):
        if  J.jobs[i].assign_mach[1] is None:
            op2_mach_id = None
        else: op2_mach_id = J.jobs[i].assign_mach[1]+1
        op1_mach_id = J.jobs[i].assign_mach[0]+1
        op1_c_time = round(J.jobs[i].end_time[0], 3)
        op2_c_time = round(J.jobs[i].end_time[1], 3)
        machine.append([op1_mach_id, op2_mach_id])
        completion_time.append([op1_c_time, op2_c_time])
    assert len(machine) == len(completion_time) == n
    return machine, completion_time

In [312]:
R = make_result(Results[0]['J']) # TA要的格式 

In [314]:
import joblib 
filepath = './OR_Case2_LSTratio_findhole_sol.pkl'
R = [make_result(Results[i]['J']) for i in range(5)]
joblib.dump(R, filepath)

['./OR_Case2_LSTratio_findhole_sol.pkl']

## Checker and Result Generated

In [319]:
import joblib 
Ans = joblib.load(filepath)

In [321]:
for i in range(5):
    res = checker(Ans[i], instances[i])
    if res:
        tardy, makespan, sch = res
        print(f'Testcase {i+1} passed.')
        print(f'tardy number: {len(tardy)}, tardies: {tardy}, makespan: {makespan}')
        # print(*sch, sep = '\n')
    else: print(f'Testcase {i+1} failed.')
    print('=================')

Testcase 1 passed.
tardy number: 0, tardies: [], makespan: 7.1
Testcase 2 passed.
tardy number: 1, tardies: [6], makespan: 8.6
Testcase 3 passed.
tardy number: 2, tardies: [1, 2], makespan: 11.9
Testcase 4 passed.
tardy number: 5, tardies: [1, 2, 3, 9, 10], makespan: 22.01
Testcase 5 passed.
tardy number: 8, tardies: [2, 4, 6, 7, 10, 14, 18, 19], makespan: 29.2
